In [1]:
import keras
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import SGD, Adam
from keras.models import Sequential,Input,Model,load_model
from keras.layers import Dense, Dropout, Flatten, GaussianNoise
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import TensorBoard
from keras import regularizers

Using TensorFlow backend.


In [3]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img

def preprocess_input_vgg(x):
    """Wrapper around keras.applications.vgg16.preprocess_input()
    to make it compatible for use with keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument.
    
    Parameters
    ----------
    x : a numpy 3darray (a single image to be preprocessed)
    
    Note we cannot pass keras.applications.vgg16.preprocess_input()
    directly to to keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument because the former expects a
    4D tensor whereas the latter expects a 3D tensor. Hence the
    existence of this wrapper.
    
    Returns a numpy 3darray (the preprocessed image).
    
    """
    from keras.applications.inception_v3 import preprocess_input
    X = np.expand_dims(x, axis=0)
    X = preprocess_input(X)
    return X[0]

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\bkl\train',
                                                    target_size=[140, 140],
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)
validation_generator = validation_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\bkl\test',
                                                              target_size=[140, 140],
                                                              batch_size=32,
                                                              class_mode='categorical')

Found 1721 images belonging to 2 classes.
Found 444 images belonging to 2 classes.


In [4]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
inc_model = InceptionV3(weights='imagenet', include_top=False)

# get layers and add average pooling layer
x = inc_model.output
x = GlobalAveragePooling2D()(x)

# add fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
# add output layer
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=inc_model.input, outputs=predictions)

# freeze pre-trained model area's layer
for layer in inc_model.layers:
    layer.trainable = False

# update the weight that are added
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'] )
model.fit_generator(train_generator,
                    steps_per_epoch=64,
                    nb_epoch=1,
                    validation_data=validation_generator,
                    validation_steps=32)

# choose the layers which are updated by training
#layer_num = len(model.layers)
#for layer in model.layers[:248]:
#    layer.trainable = False

for layer in model.layers:
    layer.trainable = True

# training
model.compile(optimizer=SGD(lr=1e-4, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

C:\Users\7\Anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:30: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\7\Anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=64, validation_data=<keras_pre..., validation_steps=32, epochs=1)`


Epoch 1/1
64/64 [==============================] - 70s 1s/step - loss: 1.5861 - acc: 0.5386 - val_loss: 0.8790 - val_acc: 0.5128


In [11]:
history = model.fit_generator(train_generator,
                    steps_per_epoch=64,
                    nb_epoch=10,
                    shuffle = True,
                    validation_data=validation_generator,
                    validation_steps=16)

C:\Users\7\Anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
C:\Users\7\Anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=64, shuffle=True, validation_data=<keras_pre..., validation_steps=16, epochs=10)`
  


Epoch 1/10
64/64 [==============================] - 30s 469ms/step - loss: 0.3526 - acc: 0.8493 - val_loss: 0.5218 - val_acc: 0.7795
Epoch 2/10
64/64 [==============================] - 29s 446ms/step - loss: 0.3750 - acc: 0.8409 - val_loss: 0.5018 - val_acc: 0.7854
Epoch 3/10
64/64 [==============================] - 29s 451ms/step - loss: 0.3446 - acc: 0.8383 - val_loss: 0.5322 - val_acc: 0.7559
Epoch 4/10
64/64 [==============================] - 29s 452ms/step - loss: 0.3474 - acc: 0.8359 - val_loss: 0.4760 - val_acc: 0.7953
Epoch 5/10
64/64 [==============================] - 28s 444ms/step - loss: 0.3492 - acc: 0.8488 - val_loss: 0.5098 - val_acc: 0.7776
Epoch 6/10
64/64 [==============================] - 31s 482ms/step - loss: 0.3383 - acc: 0.8642 - val_loss: 0.5298 - val_acc: 0.7736
Epoch 7/10
64/64 [==============================] - 29s 448ms/step - loss: 0.3168 - acc: 0.8594 - val_loss: 0.5077 - val_acc: 0.7817
Epoch 8/10
64/64 [==============================] - 26s 412ms/step - 

In [12]:
import pandas as pd

predict_gen = validation_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\bkl\test',
                                                              target_size=[140,140],
                                                              batch_size=444,
                                                              class_mode='categorical')

X_val_sample, res = next(predict_gen)
y_pred = model.predict(X_val_sample)

CM = []
for i in range(2):
    CM.append([0, 0])

Found 444 images belonging to 2 classes.


In [13]:
for x, y in zip(res, y_pred):
    CM[np.argmax(x)][np.argmax(y)] = CM[np.argmax(x)][np.argmax(y)] + 1

for line in CM:
    print(line)

model.evaluate_generator(validation_generator, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)

[197, 23]
[77, 147]
14/14 [==============================] - 4s 255ms/step


[0.5043626304145332, 0.7747747763857111]

In [15]:
onedtrue = []
onedpred = []
for true, pred in zip(res, y_pred):
    onedtrue.append(np.argmax(true))
    onedpred.append(np.argmax(pred))

In [16]:
from sklearn.metrics import classification_report
target_names = ['bcc', 'bcc-non']
print(classification_report(onedtrue, onedpred, target_names=target_names))

              precision    recall  f1-score   support

         bcc       0.72      0.90      0.80       220
     bcc-non       0.86      0.66      0.75       224

   micro avg       0.77      0.77      0.77       444
   macro avg       0.79      0.78      0.77       444
weighted avg       0.79      0.77      0.77       444



In [10]:
model.save('bkl-7838-40epc.h5')

# Ensemble

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import SGD, Adam
from keras.models import Sequential,Input,Model,load_model
from keras.layers import Dense, Dropout, Flatten, GaussianNoise
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import TensorBoard
from keras import regularizers


Using TensorFlow backend.


In [3]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img

def preprocess_input_vgg(x):
    """Wrapper around keras.applications.vgg16.preprocess_input()
    to make it compatible for use with keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument.
    
    Parameters
    ----------
    x : a numpy 3darray (a single image to be preprocessed)
    
    Note we cannot pass keras.applications.vgg16.preprocess_input()
    directly to to keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument because the former expects a
    4D tensor whereas the latter expects a 3D tensor. Hence the
    existence of this wrapper.
    
    Returns a numpy 3darray (the preprocessed image).
    
    """
    from keras.applications.inception_v3 import preprocess_input
    X = np.expand_dims(x, axis=0)
    X = preprocess_input(X)
    return X[0]

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\akiec\train',
                                                    target_size=[140, 140],
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)
validation_generator = validation_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\akiec\test',
                                                              target_size=[140, 140],
                                                              batch_size=32,
                                                              class_mode='categorical')

Found 541 images belonging to 2 classes.
Found 136 images belonging to 2 classes.


In [4]:
model = load_model(r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\ensemble\bkl-7838-40epc.h5')

In [5]:
predictions = []
for k in range(506):
    predictions.append([])

In [6]:
import pandas as pd

predict_gen = validation_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\unbalanced\test',
                                                              target_size=[140,140],
                                                              batch_size=506,
                                                              shuffle = False,
                                                              class_mode='categorical')

X_val_sample, res = next(predict_gen)
y_pred = model.predict(X_val_sample)


Found 506 images belonging to 6 classes.


In [7]:
for index,y in enumerate(y_pred):
    predictions[index].append(y[0])

In [8]:
onedtrue = []
onedpred = []
for true, pred in zip(res, y_pred):
    onedtrue.append(np.argmax(true))
    onedpred.append(np.argmax(pred))

In [9]:
from sklearn.metrics import confusion_matrix
confusion_matrix(onedtrue, onedpred)

array([[27, 39,  0,  0,  0,  0],
       [36, 67,  0,  0,  0,  0],
       [89, 14,  0,  0,  0,  0],
       [49, 54,  0,  0,  0,  0],
       [17, 86,  0,  0,  0,  0],
       [ 1, 27,  0,  0,  0,  0]], dtype=int64)

In [10]:
import pickle
with open("pred_bkl.txt", "wb") as fp:   #Pickling
    pickle.dump(predictions, fp)